In [ ]:
!pip install pandas numpy scikit-learn shap matplotlib seaborn

import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    roc_auc_score
)
!pip install shap
import shap

In [ ]:
# Inspect first rows and column names
print("Columns:", df.columns.tolist())
print("First 5 rows:\n", df.head())


Columns: ['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'label']
First 5 rows:
    flow_duration  Header_Length  Protocol Type  Duration         Rate  \
0       0.000000          54.00           6.00     64.00     0.329807   
1       0.000000          57.04           6.33     64.00     4.290556   
2       0.000000           0.00           1.00     64.00    33.396799   
3       0.328175       76175.00          17.00     64.00  4642.133010   
4       0.117320         101.73           6.11     65.91     6.202211   

       

In [ ]:

df = pd.read_csv("/content/IoT_Intrusion.csv")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Target distribution:\n", df['label'].value_counts())

Shape: (139155, 47)
Columns: ['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'label']
Target distribution:
 label
DDoS-ICMP_Flood            21412
DDoS-UDP_Flood             16107
DDoS-TCP_Flood             13478
DDoS-PSHACK_Flood          12398
DDoS-RSTFINFlood           12007
DDoS-SYN_Flood             11953
DDoS-SynonymousIP_Flood    10697
DoS-UDP_Flood               9966
DoS-TCP_Flood               7908
DoS-SYN_Flood               6056
BenignTraffic               3203
Mirai-greeth_flood          2917
Mirai-udpplain   

In [ ]:

print("Missing values before:", df.isnull().sum().sum())
df = df.drop_duplicates()
df = df.dropna()
print("After cleaning:", df.shape)

Missing values before: 7
After cleaning: (137789, 47)


In [ ]:

le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [ ]:
X = df.drop(columns=['label'])
y = df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

model building

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

y_test_bin = label_binarize(y_test, classes=model.classes_)

roc_auc = roc_auc_score(y_test_bin, y_proba, multi_class="ovr", average="macro")
print("ROC-AUC (macro):", roc_auc)


ROC-AUC (macro): nan


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
mpe = np.mean((y_test - y_pred) / y_test) * 100
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("MAPE:", mape)
print("MPE:", mpe)
print("R² Score:", r2)


MSE: 3.0963785470643734
RMSE: 1.7596529621105332
MAE: 0.15197038972349228
MAPE: inf
MPE: -inf
R² Score: 0.9092398244119508


SHAP implementation

In [ ]:
import shap

X_sample = X_test.sample(200, random_state=42)

explainer = shap.Explainer(model, X_sample)
shap_values = explainer(X_sample)

shap.summary_plot(shap_values, X_sample)
shap.plots.waterfall(shap_values[0])
